In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sys

from sklearn.metrics.pairwise import euclidean_distances

In [2]:
clean_df = pd.read_csv('data/clean_data.csv')
clean_df

,Names,Genders,Age Ranges,Specialization,Categories,Math Tutor,Science Tutor,Social Tutor,Technology Tutor,Music Tutor,...,CSN1,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5
0,Novia Rizki Wulandari,1,17-25,Flutter,Technology,0,0,0,1,0,...,5,4,4,5,5,4,4,2,4,4
1,Nabhan Nabilah,1,17-25,Patung,Arts,0,0,0,0,0,...,5,2,5,5,5,2,2,5,5,2
2,Herlina Kusyanuri Putri,1,17-25,Digital Arts,Arts,0,0,0,0,0,...,5,5,5,5,5,4,3,3,4,3
3,Rifdah Alyaa,1,17-25,Web Development,Technology,0,0,0,1,0,...,2,3,2,3,4,3,2,3,4,3
4,Salsabilla,1,17-25,Krita,Arts,0,0,0,0,0,...,4,3,3,4,4,2,2,2,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Usamah,0,17-25,Pendidikan Kewarganegaraan,Social,0,0,1,0,0,...,5,5,4,5,5,4,4,5,5,5
97,Helena,1,17 - 25,Photoshop,Multimedia,0,0,0,0,0,...,4,3,4,4,4,3,4,2,5,4
98,Deva,1,26-34,Sejarah Dunia,Social,0,0,1,0,0,...,4,4,4,3,4,3,4,2,4,3
99,Dzaky,0,17-25,Biologi,Science,0,1,0,0,0,...,5,5,5,5,5,5,5,5,4,5


In [3]:
# Check that we not missing any value after combine columns
print('Is there any missing value? ', clean_df.isnull().values.any())
print('How many missing values? ', clean_df.isnull().values.sum())

Is there any missing value?  False
How many missing values?  0


In [4]:
# Copy origin user data into match data to use in cluster
match_df = clean_df.copy()

In [5]:
# Drop Column that not use for clustering match
match_df.drop([
    'Names',
    'Age Ranges',
    'Specialization',
    'Categories',
], axis=1, inplace=True)

match_df

,Genders,Math Tutor,Science Tutor,Social Tutor,Technology Tutor,Music Tutor,Arts Tutor,Multimedia Tutor,Language Tutor,EXT1,...,CSN1,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5
0,1,0,0,0,1,0,0,0,0,4,...,5,4,4,5,5,4,4,2,4,4
1,1,0,0,0,0,0,1,0,0,1,...,5,2,5,5,5,2,2,5,5,2
2,1,0,0,0,0,0,1,0,0,3,...,5,5,5,5,5,4,3,3,4,3
3,1,0,0,0,1,0,0,0,0,3,...,2,3,2,3,4,3,2,3,4,3
4,1,0,0,0,0,0,1,0,0,2,...,4,3,3,4,4,2,2,2,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0,0,0,1,0,0,0,0,0,4,...,5,5,4,5,5,4,4,5,5,5
97,1,0,0,0,0,0,0,1,0,3,...,4,3,4,4,4,3,4,2,5,4
98,1,0,0,1,0,0,0,0,0,4,...,4,4,4,3,4,3,4,2,4,3
99,0,0,1,0,0,0,0,0,0,4,...,5,5,5,5,5,5,5,5,4,5


In [6]:
persona_test_result = match_df.iloc[:, -25:]
persona_test_result

,EXT1,EXT2,EXT3,EXT4,EXT5,EST1,EST2,EST3,EST4,EST5,...,CSN1,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5
0,4,2,4,4,3,3,3,1,2,3,...,5,4,4,5,5,4,4,2,4,4
1,1,3,4,3,2,5,5,5,3,2,...,5,2,5,5,5,2,2,5,5,2
2,3,3,1,1,2,2,4,4,5,3,...,5,5,5,5,5,4,3,3,4,3
3,3,4,2,1,2,3,2,2,4,3,...,2,3,2,3,4,3,2,3,4,3
4,2,4,4,2,2,2,5,4,4,3,...,4,3,3,4,4,2,2,2,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,4,4,5,4,2,2,4,4,4,4,...,5,5,4,5,5,4,4,5,5,5
97,3,4,2,2,2,5,5,4,5,4,...,4,3,4,4,4,3,4,2,5,4
98,4,4,4,4,4,2,3,2,4,3,...,4,4,4,3,4,3,4,2,4,3
99,4,5,5,5,5,1,1,1,1,4,...,5,5,5,5,5,5,5,5,4,5


In [7]:
# Convert input data to float32
data_input = np.array(persona_test_result, dtype=np.float32)

# Load model
model_path = '../personality_classification/persona_model/1'
loaded_model = tf.saved_model.load(model_path)

# Get the default signature of the model
inference_fn = loaded_model.signatures["serving_default"]

# Use models to make persona type predictions
predictions = inference_fn(tf.constant(data_input))["dense_2"]

In [8]:
predicted_clusters = np.argmax(predictions, axis=1)
persona_type = pd.DataFrame({'Persona type': predicted_clusters})
persona_type 

,Persona type
0,2
1,3
2,1
3,3
4,1
...,...
96,4
97,4
98,0
99,2


In [9]:
# Personality type column is changed to one hot encoding
persona_type = pd.get_dummies(persona_type, columns=['Persona type'])
persona_type = persona_type.astype(int)
persona_type

,Persona type_0,Persona type_1,Persona type_2,Persona type_3,Persona type_4
0,0,0,1,0,0
1,0,0,0,1,0
2,0,1,0,0,0
3,0,0,0,1,0
4,0,1,0,0,0
...,...,...,...,...,...
96,0,0,0,0,1
97,0,0,0,0,1
98,1,0,0,0,0
99,0,0,1,0,0


In [10]:
# Concatenate 'match_df' with 'persona_type'
match_df = pd.concat([match_df.iloc[:, :-25], persona_type], axis=1)
match_df

,Genders,Math Tutor,Science Tutor,Social Tutor,Technology Tutor,Music Tutor,Arts Tutor,Multimedia Tutor,Language Tutor,Persona type_0,Persona type_1,Persona type_2,Persona type_3,Persona type_4
0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
1,1,0,0,0,0,0,1,0,0,0,0,0,1,0
2,1,0,0,0,0,0,1,0,0,0,1,0,0,0
3,1,0,0,0,1,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0,0,0,1,0,0,0,0,0,0,0,0,0,1
97,1,0,0,0,0,0,0,1,0,0,0,0,0,1
98,1,0,0,1,0,0,0,0,0,1,0,0,0,0
99,0,0,1,0,0,0,0,0,0,0,0,1,0,0


In [11]:
# Concatenate 'match_df' with 'persona_type'
clean_df = pd.concat([clean_df.iloc[:, :-25], persona_type], axis=1)
clean_df

,Names,Genders,Age Ranges,Specialization,Categories,Math Tutor,Science Tutor,Social Tutor,Technology Tutor,Music Tutor,Arts Tutor,Multimedia Tutor,Language Tutor,Persona type_0,Persona type_1,Persona type_2,Persona type_3,Persona type_4
0,Novia Rizki Wulandari,1,17-25,Flutter,Technology,0,0,0,1,0,0,0,0,0,0,1,0,0
1,Nabhan Nabilah,1,17-25,Patung,Arts,0,0,0,0,0,1,0,0,0,0,0,1,0
2,Herlina Kusyanuri Putri,1,17-25,Digital Arts,Arts,0,0,0,0,0,1,0,0,0,1,0,0,0
3,Rifdah Alyaa,1,17-25,Web Development,Technology,0,0,0,1,0,0,0,0,0,0,0,1,0
4,Salsabilla,1,17-25,Krita,Arts,0,0,0,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Usamah,0,17-25,Pendidikan Kewarganegaraan,Social,0,0,1,0,0,0,0,0,0,0,0,0,1
97,Helena,1,17 - 25,Photoshop,Multimedia,0,0,0,0,0,0,1,0,0,0,0,0,1
98,Deva,1,26-34,Sejarah Dunia,Social,0,0,1,0,0,0,0,0,1,0,0,0,0
99,Dzaky,0,17-25,Biologi,Science,0,1,0,0,0,0,0,0,0,0,1,0,0


In [12]:
match_array = match_df.values
match_array.shape

(101, 14)

# Build PCA Model

## Input Size (1 Genders + 8 Categories + 5 Personality)

In [13]:
class PCAModel(tf.Module):
    def __init__(self, x_mean, components):
        super(PCAModel, self).__init__()
        self.x_mean = tf.Variable(x_mean, trainable=False)
        self.components = tf.Variable(components, trainable=False)

    @tf.function(input_signature=[tf.TensorSpec(shape=(None, 14), dtype=tf.float32)])
    def scale_features(self, x):
        x_min = tf.reduce_min(x, axis=0)
        x_max = tf.reduce_max(x, axis=0)

        # Handle columns with zero variance
        zero_variance_mask = tf.math.equal(x_min, x_max)
        non_zero_variance_mask = tf.math.logical_not(zero_variance_mask)

        # Scale the columns with non-zero variance
        x_scaled_non_zero = tf.where(non_zero_variance_mask, (x - x_min) / (x_max - x_min), x)

        # Scale the columns with zero variance
        x_scaled_zero = tf.where(zero_variance_mask, tf.zeros_like(x), x_scaled_non_zero)

        return x_scaled_zero

    @tf.function(input_signature=[tf.TensorSpec(shape=(None, 14), dtype=tf.float32)])
    def apply_pca(self, x):
        x_scaled = self.scale_features(x)
        x_centered = x_scaled - self.x_mean
        
        # Compute the covariance matrix
        covariance_matrix = tf.matmul(tf.transpose(x_centered), x_centered) / tf.cast(tf.shape(x_centered)[0], dtype=tf.float32)

        # Perform eigenvalue decomposition
        eigenvalues, eigenvectors = tf.linalg.eigh(covariance_matrix)
        
        # Sort eigenvectors based on eigenvalues
        sorted_indices = tf.argsort(eigenvalues, direction='DESCENDING')
        sorted_eigenvectors = tf.gather(eigenvectors, sorted_indices, axis=1)
        
        # Select the top k eigenvectors
        k = tf.minimum(tf.shape(sorted_eigenvectors)[1], 2)  # Choose the top 2 eigenvectors (modify as needed)
        selected_eigenvectors = sorted_eigenvectors[:, :k]
        
        # Project the centered data onto the selected eigenvectors
        x_pca = tf.matmul(x_centered, selected_eigenvectors)
        
        return x_pca

# Convert the numpy array to a TensorFlow tensor
x_tf = tf.convert_to_tensor(match_array, dtype=tf.float32)

# Perform PCA with k=2 (reduce to 2 dimensions)
x_mean = tf.reduce_mean(x_tf, axis=0)
_, _, V = tf.linalg.svd(x_tf - x_mean)
components = V[:, :2]

# Create an instance of the PCA model
pca_model = PCAModel(x_mean, components)

In [14]:
# Save the PCAModel as a SavedModel
tf.saved_model.save(
    pca_model,
    export_dir='pca_matching/4/',
    signatures={
        'serving_default': pca_model.apply_pca.get_concrete_function()
    }
)

INFO:tensorflow:Assets written to: pca_matching/4/assets


In [15]:

!saved_model_cli show --dir {'pca_matching/4/'} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['x'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 14)
        name: serving_default_x:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 2)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
The MetaGraph with tag set ['serve'] contains the following ops: {'NoOp', 'SaveV2', 'AssignVariableOp', 'Const', 'MergeV2Checkpoints', 'ShardedFilenam

## Test Load Model Directly

In [16]:
# Load the PCA model
loaded_model = tf.saved_model.load("pca_matching/4/")

In [18]:
# Generate a new data point
new_data = np.random.rand(1, 14)
new_data_tf = tf.convert_to_tensor(new_data, dtype=tf.float32)

# Predict the PCA value using the loaded model
pca_value = loaded_model.apply_pca(new_data_tf)

# Print the PCA value
print(pca_value)

tf.Tensor([[-9.1588181e-01 -1.4901161e-08]], shape=(1, 2), dtype=float32)


In [19]:
# Predict PCA Valuse Using Our Match/Users Datasets
new_data_tf = tf.convert_to_tensor(match_array, dtype=tf.float32)

# Predict the PCA value using the loaded model
pca_value = loaded_model.apply_pca(new_data_tf)

# Print the PCA value
print(pca_value)

tf.Tensor(
[[-0.21087807 -0.19615018]
 [-0.3371854   0.03751896]
 [-0.31958193 -0.36838186]
 [-0.41479906  0.15651844]
 [-0.31958193 -0.36838186]
 [-0.18966933  0.21494612]
 [-0.23268728  0.5394807 ]
 [ 0.5195013   0.4687189 ]
 [-0.14378077 -0.72342515]
 [-0.3724151   0.5158531 ]
 [-0.32939714  0.19131859]
 [ 0.7971784  -0.13279326]
 [-0.351527   -0.02372488]
 [ 0.66701734  0.64614606]
 [ 0.8536156   0.124967  ]
 [ 0.95314896 -0.00856   ]
 [-0.5665208   0.30882132]
 [-0.49694902  1.0481734 ]
 [-0.26055866 -0.8251613 ]
 [-0.5587326   0.46262097]
 [-0.13326442 -0.31514966]
 [-0.58414626  0.88546586]
 [-0.20412152 -0.56740105]
 [-0.29129684 -0.9008523 ]
 [-0.28884375 -0.2926908 ]
 [-0.18966933  0.21494612]
 [-0.26055866 -0.8251613 ]
 [-0.39719558 -0.24938236]
 [-0.39719558 -0.24938236]
 [ 0.01425166 -0.1377225 ]
 [-0.32939714  0.19131859]
 [-0.6158494  -0.10464969]
 [-0.31958193 -0.36838186]
 [-0.31960386 -0.19763808]
 [-0.661738    0.83372164]
 [-0.29131877 -0.7301086 ]
 [ 0.8536156   0.

## Build Matchmaking Feature

In [20]:
def predict_pca(match_array):
    # Load the PCA model
    loaded_model = tf.saved_model.load('pca_matching/4/')
    new_data_tf = tf.convert_to_tensor(match_array, dtype=tf.float32)

    # Predict the PCA value using the loaded model
    pca_value = loaded_model.apply_pca(new_data_tf)
    pca_value_arr = pca_value.numpy()
    
    return pca_value_arr

In [21]:
def get_guides_idx_filtered(user_id, user_df):
    # Get user destination and persona
    user_destination = user_df.loc[user_id, 'Categories']

    # Filter users with the same destination and roles value of 'guide'
    filtered_users = user_df[(user_df['Categories'] == user_destination)]

    user_indices = []
    for idx in filtered_users.index:
        user_indices.append(idx)
    user_indices = [idx for idx in user_indices if idx != user_id]
            
    return user_indices

In [22]:
def matchmaking(user_id, match_array, user_df):
    
    filtered_user_indices = get_guides_idx_filtered(user_id, user_df)
    if len(filtered_user_indices) == 0:
        print("No Results")
        sys.exit()
        
    user_indices = filtered_user_indices + [user_id]
    
    pca_data = predict_pca(match_array[user_indices])

    distances = []
    for idx in user_indices:
        # Get the distances within the cluster of the specified user

        distance_idx = euclidean_distances([pca_data[user_indices.index(user_id)]], [pca_data[user_indices.index(idx)]])
        distances.append(distance_idx)

    # Normalize the distances within the range of 0-100
    normalized_distances = 1 - distances / np.max(distances)
    scores = normalized_distances * 100
    
    # Reshape the list
    scores = np.reshape(scores, (len(user_indices)))
    
    matches = []
    for i in range(len(filtered_user_indices)):
        matches.append((filtered_user_indices[i], scores[i]))
    
    # Sort the matches based on the highest score
    matches.sort(key=lambda x: x[1], reverse=True)

    print("User Index:", user_id)
    print("Matched Peers:")
    match_idx = [user_id]
    for match_index, score in matches:
        if user_df.iloc[match_index, 2] == user_df.iloc[user_id, 2]:
            match_idx.append(match_index)
            print("Index:", match_index, "| Score:", f"{score:.2f}%")
                
    # Check if Destination and roles is correct 
    print()         
    print(user_df.iloc[match_idx, [0, 1, 3, 4]])
    
    # Check if they get same preference
    print()
    print(user_df.iloc[match_idx, 5:])

In [23]:
matchmaking(0, match_array, clean_df)

User Index: 0
Matched Peers:
Index: 3 | Score: 79.59%
Index: 81 | Score: 75.24%
Index: 95 | Score: 47.36%
Index: 28 | Score: 41.02%
Index: 94 | Score: 41.02%
Index: 34 | Score: 32.92%
Index: 41 | Score: 18.47%

                     Names  Genders     Specialization  Categories
0   Novia Rizki Wulandari         1            Flutter  Technology
3             Rifdah Alyaa        1    Web Development  Technology
81     Lili Nandita Auliya        1      Multiplatform  Technology
95                   Salma        1           HTML CSS  Technology
28        Nur Alifia Riany        1            Android  Technology
94                  Annisa        1            Laravel  Technology
34                    Zeen        0  Backend Developer  Technology
41                 Gustian        0              React  Technology

    Math Tutor  Science Tutor  Social Tutor  Technology Tutor  Music Tutor   
0            0              0             0                 1            0  \
3            0              0

In [24]:
matchmaking(58, match_array, clean_df)

User Index: 58
Matched Peers:
Index: 5 | Score: 62.22%
Index: 25 | Score: 62.22%
Index: 13 | Score: 22.86%
Index: 8 | Score: 2.44%
Index: 92 | Score: 2.44%
Index: 57 | Score: 0.00%
Index: 83 | Score: 0.00%

                 Names  Genders          Specialization  Categories
58     Alyzar Aviandi         0          Penulisan Buku  Multimedia
5              Maulani        1               Fotografi  Multimedia
25              Elma N        1                   Excel  Multimedia
13  Bayu Daru Isnandar        0            Audio Mixing  Multimedia
8                   JJ        1              Videografi  Multimedia
92                 Féi        1            Editing Foto  Multimedia
57             kartika        1        Microsoft Office  Multimedia
83          Dhea Setya        1  Videografi dan Editing  Multimedia

    Math Tutor  Science Tutor  Social Tutor  Technology Tutor  Music Tutor   
58           0              0             0                 0            0  \
5            0          